In [1]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

import h5py

import importlib
import models, utils
importlib.reload(models)
importlib.reload(utils)
from models import *
from utils import *
torch.set_float32_matmul_precision('medium')
f_train = h5py.File("../data/QG_Images/quark-gluon_train-set_n793900.hdf5","r")
f_test_1 = h5py.File("../data/QG_Images/quark-gluon_test-set_n10000.hdf5","r")
f_test_2 = h5py.File("../data/QG_Images/quark-gluon_test-set_n139306.hdf5","r")

print(f_train.keys())
print(f_test_1.keys())
print(f_test_2.keys())

Device: cuda
Device: cuda
<KeysViewHDF5 ['X_jets', 'm0', 'pt', 'y']>
<KeysViewHDF5 ['X', 'y']>
<KeysViewHDF5 ['X_jets', 'm0', 'pt', 'y']>


In [2]:
X_train = f_train.get("X_jets")
y_train = f_train.get("y")

X_test1 = f_test_1.get("X")
y_test1 = f_test_1.get("y")

X_test2 = f_test_2.get("X_jets")
y_test2 = f_test_2.get("y")
pt_test2 = f_test_2.get("pt")
m_test2 = f_test_2.get("m0")

In [3]:
print(len(X_train), len(X_test1), len(X_test2))

793900 10000 139306


In [4]:
pt_test2.shape

print(pt_test2[:10])

[107.45191  133.12431   88.46564  111.72106  110.947655 111.129745
  87.034294  90.747     93.22315  105.96356 ]


In [5]:
m_test2.shape

(139306,)

In [6]:
# visualize_average_images(X_train[:10000], y_train[:10000])
# visualize_average_images(X_train[:10000], y_train[:10000], num=1)

# visualize_diff_average_images(X_train[:10000], y_train[:10000])

In [7]:
dataset = QG_Images(images=X_test1[:5, :, :, 2], labels=y_test1)
print(dataset[0])
print(dataset[1])
print(dataset[2])

Data(x=[425, 1], y=[1], pos=[425, 2], num_nodes=425)
Data(x=[425, 1], y=[1], pos=[425, 2], num_nodes=425)
Data(x=[450, 1], y=[1], pos=[450, 2], num_nodes=450)


Processing...
Done!


In [8]:
dataset = QG_Images(images=X_test1[:5, :, :, 1], labels=y_test1)
print(dataset[0])
print(dataset[1])

Data(x=[269, 1], y=[1], pos=[269, 2], num_nodes=269)
Data(x=[432, 1], y=[1], pos=[432, 2], num_nodes=432)


Processing...
Done!


In [9]:
dataset = QG_Images(images=X_test1[:, :, :, 0], labels=y_test1)
print(len(dataset))
print(dataset[0])
print(dataset[1])
print(torch.min(dataset[:].x), torch.max(dataset[:].x))

Processing...
Done!


10000
Data(x=[59, 1], y=[1], pos=[59, 2], num_nodes=59)
Data(x=[78, 1], y=[1], pos=[78, 2], num_nodes=78)
tensor(0.0013) tensor(13.3948)


## Contrastive

In [10]:
EPOCH = 5

In [11]:
from qml_ssl.utils import vmf_kde_on_circle, pca_proj, tsne_proj, plot_training


dataset = QG_Images(images=X_test1[:, :, :, 0], labels=y_test1)
total_size = 10000
dataset = dataset[:total_size]

train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = int(0.2 * total_size)

batch_size = 32

k = 15

dataset.transform = T.Compose([TopKIntensity(k=k), KNNGroup(k=15, attr_name='x')])
sample = dataset[10]
print(sample)

print(f'Node degree: {sample.num_edges / sample.num_nodes:.2f}')
print(f'Has isolated nodes: {sample.has_isolated_nodes()}')
print(f'Has self-loops: {sample.has_self_loops()}')
print(f'Is undirected: {sample.is_undirected()}')
nx.draw(pyg.utils.to_networkx(dataset[10]))
plt.show()

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = pyg_loader.DataLoader(train_dataset, batch_size=batch_size, num_workers = 4)
val_loader = pyg_loader.DataLoader(val_dataset, batch_size=batch_size, num_workers = 4)
test_loader = pyg_loader.DataLoader(test_dataset, batch_size=batch_size, num_workers = 4)

gcn_encoder = GCN_Encoder()

Contrastive_Graph_pl = ModelPL_Contrastive(model=gcn_encoder, learning_rate=0.001)

logger = pl.loggers.CSVLogger(save_dir='logs', name='Contrastive_Graph_image_pl', version=0)

summary_callback = pl.callbacks.ModelSummary(max_depth=8)
callbacks = [summary_callback]

# embeddings, labels = generate_embeddings(Contrastive_Graph_pl, val_loader)
# pca_proj(embeddings, labels)
# # tsne_proj(embeddings, labels)
# vmf_kde_on_circle(embeddings, labels)

Processing...


In [12]:
trainer_contrastive = pl.Trainer(max_epochs=EPOCH, 
                            devices="auto",
                            callbacks=callbacks,
                            logger=logger,)

train_result = trainer_contrastive.fit(Contrastive_Graph_pl, train_dataloaders=train_loader, val_dataloaders=val_loader)
test_result = trainer_contrastive.test(dataloaders=test_loader)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/duydl/Miniconda3/envs/py310_qml/lib/python3.10/site-packages/lightning_fabric/loggers/csv_logs.py:268: Experiment logs directory logs/Contrastive_Graph_image_pl/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/home/duydl/Miniconda3/envs/py310_qml/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory logs/Contrastive_Graph_image_pl/version_0/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                          | Type                 | Params | Mode 
--------------------------------------------------------------------------------
0  |

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]tensor([[1.0000, 0.9978, 0.9944,  ..., 0.9951, 0.9998, 0.9964],
        [0.9978, 1.0000, 0.9992,  ..., 0.9994, 0.9988, 0.9998],
        [0.9944, 0.9992, 1.0000,  ..., 1.0000, 0.9961, 0.9998],
        ...,
        [0.9951, 0.9994, 1.0000,  ..., 1.0000, 0.9967, 0.9999],
        [0.9998, 0.9988, 0.9961,  ..., 0.9967, 1.0000, 0.9978],
        [0.9964, 0.9998, 0.9998,  ..., 0.9999, 0.9978, 1.0000]],
       device='cuda:0')
tensor([[1.0000, 0.9978, 0.9944,  ..., 0.9951, 0.9998, 0.9964],
        [0.9978, 1.0000, 0.9992,  ..., 0.9994, 0.9988, 0.9998],
        [0.9944, 0.9992, 1.0000,  ..., 1.0000, 0.9961, 0.9998],
        ...,
        [0.9951, 0.9994, 1.0000,  ..., 1.0000, 0.9967, 0.9999],
        [0.9998, 0.9988, 0.9961,  ..., 0.9967, 1.0000, 0.9978],
        [0.9964, 0.9998, 0.9998,  ..., 0.9999, 0.9978, 1.0000]],
       device='cuda:0')
{'loss': {'losses': tensor([2.7674, 2.7715, 2.7690, 2.7712, 2.7739, 2.7670, 2.7699, 2.773

AttributeError: 'MeanReducer' object has no attribute 'assert_sizes_pos_pair'

In [ ]:
embeddings, labels = generate_embeddings(Contrastive_Graph_pl, val_loader)
pca_proj(embeddings, labels)
# tsne_proj(embeddings, labels)
vmf_kde_on_circle(embeddings, labels)

In [ ]:
pretrained_model = GCN_Encoder()
# checkpoint_path = './logs/Contrastive_Graph_pl/version_0/checkpoints/epoch=14-step=14070.ckpt'
pretrained_model.load_state_dict(Contrastive_Graph_pl.model.state_dict())

num_classes = 2  # Adjust this based on your dataset

# Create an instance of the LinearProbePL module
linear_probe_model = LinearProbePL(pretrained_model=pretrained_model, num_classes=num_classes, learning_rate=0.001)

trainer_linear_probe = pl.Trainer(max_epochs=EPOCH, 
                            devices="auto",
                            callbacks=callbacks,
                            logger=logger,)

train_result = trainer_linear_probe.fit(linear_probe_model, train_dataloaders=train_loader, val_dataloaders=val_loader)
test_result = trainer_linear_probe.test(linear_probe_model, dataloaders=test_loader)

In [ ]:
metrics_file = "./logs/Contrastive_Graph_image_pl/version_0/metrics.csv"

plot_metrics_from_csv(metrics_file)


In [ ]:
metrics_file = 'logs/graph_image_pl_2_100/version_0/metrics.csv' 

plot_metrics_from_csv(metrics_file)